In [ ]:
%pip install packaging requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse
import logging

# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_html_with_requests(url):
    # Extrair domínio e caminho da URL
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    path = parsed_url.path

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,ru;q=0.6,es;q=0.5',
        'Cookie': 'ajs_anonymous_id=a03ba2e0-6658-4732-bca0-5c82a729610d; blueID=158f8c36-f884-4eeb-9dd2-41ddb36954f2; _fbp=fb.2.1719565497536.945089496421286543; _tt_enable_cookie=1; _ttp=yDIqAjbcrTkStMgfYKWUtQqV7qI; _scid=fe8f87fb-9164-4128-931f-6db8349a5902; _pin_unauth=dWlkPU16QmxZV015TVRndE16QmhOUzAwWmpRekxUZzROakF0Wm1VeFlUSTROREE1WXpJeg; _hjSessionUser_257734=eyJpZCI6IjZlYmVkNTczLWRlNDItNTljNy1iYjBkLWRjMzQxYzg1ZThkZiIsImNyZWF0ZWQiOjE3MTk1NjU0OTc1ODksImV4aXN0aW5nIjp0cnVlfQ==; __utmz=other; blueULC=blue; OptanonAlertBoxClosed=2024-06-28T10:07:39.896Z; _gac_UA-58792402-1=1.1719578340.CjwKCAjwvvmzBhA2EiwAtHVrbx5PuZ-Y6ZmDRgxSQAE3bhfdMkfQ6qO_h_InopnIh9t314DXR2UOdhoCJPMQAvD_BwE; origem=adwords; _rtbhouse_source_=AdWords; anonymousClient=true; _gcl_aw=GCL.1719608537.CjwKCAjwvvmzBhA2EiwAtHVrb7a4Gf1uUaMKzAcQ623DOKsXA0ynRueURMbfhKPxXrTvmq46s3W5ZBoCXV8QAvD_BwE; _gcl_gs=2.1.k1$i1719608535; _gid=GA1.3.1821486782.1720470235; _ScCbts=%5B%5D; petzCarrinho=true; JSESSIONID=9E56A4F6251336BC2EEC9640328DF1CE; _hjSession_257734=eyJpZCI6ImRmYjFhNzkxLTcwMDAtNDc5ZC1hZDA2LTE4NGYwZGMwOTQ3MSIsImMiOjE3MjA2MzEyODk1MzEsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; OptanonConsent=isGpcEnabled=0&datestamp=Wed+Jul+10+2024+14%3A33%3A28+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=202404.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=5d44d058-e0ca-4634-af6a-6ae970855f35&interactionCount=2&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0003%3A1%2CC0002%3A1%2CC0004%3A1&AwaitingReconsent=false&intType=1&geolocation=BR%3BRJ; AWSALB=hZab3iIPYsD0MIUknEJjbESldllV2XbXHeW6ABubXDqQC1w+fB9ShoCYGJIR2z0ndrcA5n/bdu5qP2SU+sm1uOdvT/LotUC6UCm2meSI5Mm4DIVUR9N7D5oqkEID; AWSALBCORS=hZab3iIPYsD0MIUknEJjbESldllV2XbXHeW6ABubXDqQC1w+fB9ShoCYGJIR2z0ndrcA5n/bdu5qP2SU+sm1uOdvT/LotUC6UCm2meSI5Mm4DIVUR9N7D5oqkEID; _ga_JB2JHD7FCJ=GS1.1.1720631306.23.1.1720632809.57.0.0; _ga=GA1.1.604824160.1719565115; _uetsid=006ad4a03d6811ef82e9db6f4cebf44a; _uetvid=7e097870352d11ef8996917c57f63d87; _scid_r=fe8f87fb-9164-4128-931f-6db8349a5902; _derived_epik=dj0yJnU9OVpXTXl0T0xlTmRISjQ3aGwtVFQ1SWZjVmJuT0JiaFkmbj1zc3Z6NkNFQXNmdnZHWmFDdEpIX093Jm09MSZ0PUFBQUFBR2FPeGVzJnJtPTEmcnQ9QUFBQUFHYU94ZXMmc3A9Mg; cto_bundle=Eb4Xml9POEdtRUtaTE12dTVMYjVqT2xtMDIxaDE5blJXQWRHRkZXVEJlcUdNOTdFRGNwdGtMeWslMkZGVnBENzMwM0JYMDVmQkdIeVQ4NW1qMW9laEslMkZRQnl3dzElMkZPWFUlMkZmTFJxNGdLeXUlMkZTMTRqZyUyRndXSFZ4dEsxS1hyMk1ZeGpOQVU4bmU1bkFZY1lwcHp2M3NVZ1FWQWNmYmRYS0RIc1N1ZzFBcWRZcWNlOVcxRkVaeVZGd0YzQUxFMWglMkZheXhJa0cyJTJGdUg0NiUyQk92VCUyRnVZZmtxQWN6QVAlMkZzdyUzRCUzRA',
        'Priority': 'u=0, i',
        'Referer': f"{domain}/marcas",
        'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }

    logging.info(f"Fazendo requisição para {url}")
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        logging.info(f"Requisição bem-sucedida para {url}")
        return response.text
    else:
        logging.error(f"Erro ao obter HTML com requests: {response.status_code}")
        return None

# Função para extrair categorias recursivamente
def extract_categories(url):
    htmlcontent = get_html_with_requests(url)
    if htmlcontent is None:
        return []
    
    soup = BeautifulSoup(htmlcontent, 'html.parser')
    categories = []
    ul_element = soup.find('ul', {'data-id': '-4'})
    
    if ul_element:
        li_elements = ul_element.find_all('li', class_='show')
        for li in li_elements:
            category_name = li.find('span').text.strip()
            category_link = li.find('a')['href']
            logging.info(f"Extraindo categoria: {category_name} - {category_link}")
            subcategories = extract_categories(category_link)
            category = {
                'name': category_name,
                'link': category_link,
                'subcategories': subcategories
            }
            categories.append(category)
    
    return categories

# Função principal para organizar as categorias principais e subcategorias
def main():
    url = 'https://www.petz.com.br/casa-e-jardim'
    parsed_url = urlparse(url)
    initial_category_name = parsed_url.path.strip('/').capitalize()
    
    logging.info(f"Categoria inicial: {initial_category_name}")
    
    categories = extract_categories(url)
    
    main_category = {
        'name': initial_category_name,
        'link': url,
        'subcategories': categories
    }
    
    # Salvar a categoria principal e suas subcategorias em um arquivo JSON
    with open(f'categories_{initial_category_name}.json', 'w', encoding='utf-8') as f:
        json.dump(main_category, f, indent=2, ensure_ascii=False)
        logging.info("Categorias salvas em 'categorias.json'")

if __name__ == "__main__":
    main()